In [15]:
!pip install pyTigerGraph
!pip install -q jupyter-dash
!pip install python-dotenv
!pip install dash-cytoscape
!pip install dash-bootstrap-components

You should consider upgrading via the 'C:\Repository\Tiger-Graph-LoL-Champion-Recommender\env\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Repository\Tiger-Graph-LoL-Champion-Recommender\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Repository\Tiger-Graph-LoL-Champion-Recommender\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Repository\Tiger-Graph-LoL-Champion-Recommender\env\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Repository\Tiger-Graph-LoL-Champion-Recommender\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [9]:
# Imports
import pandas as pd
import numpy as np
import pyTigerGraph as tg
import dash
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output
import dash_cytoscape as cyto
import dash_bootstrap_components as dbc

### Open Connection to Graph

In [2]:
# Connection Params
hostname = 'https://leaguerecommender.i.tgcloud.io/'
username = 'tigergraph'
password = 'tigergraph'
graphname = 'ChampionRecommendation'

In [3]:
conn = tg.TigerGraphConnection(host=hostname, username=username, password=password, graphname=graphname)
conn.apiToken = conn.getToken(conn.createSecret())

print("Connected!")

Connected!


In [12]:
comms = conn.runInstalledQuery("GetAllRecs", params={"mainChampion": 'Ziggs'})
comms

[{'@@edgeList': [{'e_type': 'top_rec',
    'from_id': 'Ziggs',
    'from_type': 'champion',
    'to_id': 'Teemo',
    'to_type': 'champion',
    'directed': True,
    'attributes': {'dist': 0.13446}},
   {'e_type': 'top_rec',
    'from_id': 'Ziggs',
    'from_type': 'champion',
    'to_id': 'Garen',
    'to_type': 'champion',
    'directed': True,
    'attributes': {'dist': 0.15786}},
   {'e_type': 'top_rec',
    'from_id': 'Ziggs',
    'from_type': 'champion',
    'to_id': 'Malphite',
    'to_type': 'champion',
    'directed': True,
    'attributes': {'dist': 0.18634}},
   {'e_type': 'jg_rec',
    'from_id': 'Ziggs',
    'from_type': 'champion',
    'to_id': 'Nunu',
    'to_type': 'champion',
    'directed': True,
    'attributes': {'dist': 0.11576}},
   {'e_type': 'jg_rec',
    'from_id': 'Ziggs',
    'from_type': 'champion',
    'to_id': 'Diana',
    'to_type': 'champion',
    'directed': True,
    'attributes': {'dist': 0.15209}},
   {'e_type': 'jg_rec',
    'from_id': 'Ziggs',
   

### Create Functions to Call Queries

In [4]:
def getNetwork(main_champion):
  comms = conn.runInstalledQuery("GetAllRecs", params={"mainChampion": main_champion})[0]['@@edgeList']

  color_map = {'top_rec':'yellow', 'jg_rec': 'green', 'mid_rec': 'blue', 'bot_rec': 'red', 'sup_rec': 'purple'}
  vertices = {}
  els = []
  
  for entry in comms:
    if 'reverse' in entry['e_type']:
      continue

    # Get to and from nodes
    source = entry['from_id']
    target = entry['to_id']

    # Add initial mainChampion
    if source not in vertices:
      if source == main_champion:
        els.append({'data': {'id': source, 'label': source}, 'classes': 'orange'})

    # Add our target if not already present
    if target not in vertices:
      els.append({'data': {'id': target, 'label': target}, 'classes':color_map[entry['e_type']]})

    # Add from and to
    els.append({'data': {'source': source, 'target': target}})
  
  stylesheet = [{'selector': 'node',
                              'style': {
                              'content': 'data(label)'
                              }
                          }]
  
  for color in color_map.values():
    stylesheet.append({
                      'selector': f'.{color}',
                      'style': {
                          'background-color': color,
                        }
                      }
    )
  stylesheet.append({
    'selector': '.orange',
    'style': {
      'background-color': 'orange'
    }
  })

  network = cyto.Cytoscape(
                  id='cytoscape',
                  elements=els,
                  layout={'name': 'concentric'},
                  stylesheet=stylesheet,

                style={'width': '100%', 'height': '500px', 'margin-left':0}
              )

  return network

### Create Dashboard

In [16]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [17]:
@app.callback(Output('cytoscape', 'layout'),
              Input('dropdown-update-layout', 'value'))
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

In [18]:
app.layout = html.Div(children=[
    html.P(children='Hello Dash'),
    dcc.Dropdown(
        id='dropdown-update-layout',
        value='concentric',
        clearable=False,
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['grid', 'random', 'circle', 'cose', 'concentric']
        ]
    ),
    dbc.Row(getNetwork('Ziggs'))
])

app.run_server(mode='external', debug=True)

Dash app running on http://127.0.0.1:8050/
